In [1]:
import pandas as pd
import os
from google.colab import drive 
import numpy as np
import random
drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/Shared drives/Financial Modeling and Testing/Project 1/')

Mounted at /content/drive


# Load Existing Dataframe

In [2]:
df = pd.read_csv('Data/finalDataframe_wBankruptcyWithinIndicator.csv')
df['date_month'] = pd.to_datetime(df['date_month']).dt.to_period('m')
df['last_date_existance'] = pd.to_datetime(df['last_date_existance']).dt.to_period('m')
df.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (27,28) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,PERMNO,date_month,GVKEY,cik,conm,atq,ceqq,cheq,ltq,niq,dlrsn,dldte,PRC,RET,SHROUT,CFACPR,vwretdSP500,totvalSP500,NITA,NIMTA,TLTA,TLMTA,EXRET,RSIZE,CASHMTA,SIGMA,effectiveDate_Bloomberg_date_month,fileDate_AuditAnalytics_month,bankruptcyIndicator,bankruptcyDate,last_date_existance,bankruptFirm,bankruptcyWithin3Months,bankruptcyWithin6Months,bankruptcyWithin12Months,bankruptcyWithin24Months,bankruptcyWithin60Months
0,0,10000,1986-02,13007.0,NaN,OPTIMUM MANUFACTURING -CL A,1.265,0.689,0.743,0.576,-0.132,10.0,1989-09-30,-3.25000,-0.257143,3680.0,1.0,0.076525,1.589623e+12,0.000110,0.000011,-0.000482,-0.000048,NaN,-7.523798e-09,-0.000062,NaN,NaN,NaN,0,1989-09,1987-05,0,0,0,0,0,0
1,1,10000,1986-03,13007.0,NaN,OPTIMUM MANUFACTURING -CL A,1.265,0.689,0.743,0.576,-0.132,10.0,1989-09-30,-4.43750,0.365385,3680.0,1.0,0.055832,1.671574e+12,0.000081,0.000008,-0.000353,-0.000035,0.495175,-9.769236e-09,-0.000046,NaN,NaN,NaN,0,1989-09,1987-05,0,0,0,0,0,0
2,2,10000,1986-04,13007.0,NaN,OPTIMUM MANUFACTURING -CL A,1.265,0.689,0.743,0.576,-0.132,10.0,1989-09-30,-4.00000,-0.098592,3793.0,1.0,-0.013348,1.642761e+12,0.000087,0.000009,-0.000380,-0.000038,0.759830,-9.235672e-09,-0.000049,NaN,NaN,NaN,0,1989-09,1987-05,0,0,0,0,0,0
3,3,10000,1986-05,13007.0,NaN,OPTIMUM MANUFACTURING -CL A,1.284,0.578,0.395,0.706,-0.484,10.0,1989-09-30,-3.10938,-0.222656,3793.0,1.0,0.055326,1.724749e+12,0.000411,0.000041,-0.000599,-0.000060,0.773142,-6.838025e-09,-0.000033,0.718449,NaN,NaN,0,1989-09,1987-05,0,0,0,0,0,0
4,4,10000,1986-06,13007.0,NaN,OPTIMUM MANUFACTURING -CL A,1.284,0.578,0.395,0.706,-0.484,10.0,1989-09-30,-3.09375,-0.005025,3793.0,1.0,0.015744,1.748550e+12,0.000413,0.000041,-0.000602,-0.000060,0.680049,-6.711042e-09,-0.000034,0.520684,NaN,NaN,0,1989-09,1987-05,0,0,0,0,0,0


# Prices

In CRSP, the variable for closing day prices is price (PRC).  

If the closing price is not available on any given trading day, the number in the price field has a negative sign to indicate that it is a bid/ask average and not an actual closing price. It is important to note that in this field the negative sign is a symbol and that the value of the bid/ask average is not negative.

If neither closing price nor bid/ask average is available on a date, PRC is set to zero. In a monthly database, price is the price on the last trading date of the month. The price series begins the first month-end after the security begins trading and ends the last complete month of trading.

Therfore, we need to adjust all prices to reflect the current trading price of the security. Our logic for adjusting prices will be as follows:
1. Create Adjusted Closing Prices ($PRC * CFACPR$)
2. Replace negative prices or 0 prices with the last available Closing Month Price.


In [9]:
df2 = df.copy()

## Calculate Adjusted Closing Prices

In [10]:
df2['AdjPRC'] = df2['PRC'] * df2['CFACPR']

## Replace negative prices or 0 prices

In [11]:
import timeit
start = timeit.default_timer()

# Replace all negative prices with 0
df2.loc[df2['AdjPRC']<0, 'AdjPRC'] = 0

# Replace all 0 prices with NaN
df2.loc[df2['AdjPRC']==0, 'AdjPRC'] = np.nan

# Forward all prices
temp = df2.set_index(['date_month', 'PERMNO'])
temp2 = temp['AdjPRC'].groupby(level=1).transform(lambda AdjPRC: AdjPRC.ffill()).reset_index()

df2 = df2.reset_index()
df2 = df2.drop(['AdjPRC'], axis=1)
df2 = pd.merge(df2,
               temp2,
               how='inner',
               left_on=['PERMNO', 'date_month'],
               right_on=['PERMNO', 'date_month']
               )

stop = timeit.default_timer()
print('Time: ', stop - start)

df2.head()

Time:  46.80559066000001


,index,Unnamed: 0,PERMNO,date_month,GVKEY,cik,conm,atq,ceqq,cheq,ltq,niq,dlrsn,dldte,PRC,RET,SHROUT,CFACPR,vwretdSP500,totvalSP500,NITA,NIMTA,TLTA,TLMTA,EXRET,RSIZE,CASHMTA,SIGMA,effectiveDate_Bloomberg_date_month,fileDate_AuditAnalytics_month,bankruptcyIndicator,bankruptcyDate,last_date_existance,bankruptFirm,bankruptcyWithin3Months,bankruptcyWithin6Months,bankruptcyWithin12Months,bankruptcyWithin24Months,bankruptcyWithin60Months,AdjPRC
0,0,0,10000,1986-02,13007.0,NaN,OPTIMUM MANUFACTURING -CL A,1.265,0.689,0.743,0.576,-0.132,10.0,1989-09-30,-3.25000,-0.257143,3680.0,1.0,0.076525,1.589623e+12,0.000110,0.000011,-0.000482,-0.000048,NaN,-7.523798e-09,-0.000062,NaN,NaN,NaN,0,1989-09,1987-05,0,0,0,0,0,0,NaN
1,1,1,10000,1986-03,13007.0,NaN,OPTIMUM MANUFACTURING -CL A,1.265,0.689,0.743,0.576,-0.132,10.0,1989-09-30,-4.43750,0.365385,3680.0,1.0,0.055832,1.671574e+12,0.000081,0.000008,-0.000353,-0.000035,0.495175,-9.769236e-09,-0.000046,NaN,NaN,NaN,0,1989-09,1987-05,0,0,0,0,0,0,NaN
2,2,2,10000,1986-04,13007.0,NaN,OPTIMUM MANUFACTURING -CL A,1.265,0.689,0.743,0.576,-0.132,10.0,1989-09-30,-4.00000,-0.098592,3793.0,1.0,-0.013348,1.642761e+12,0.000087,0.000009,-0.000380,-0.000038,0.759830,-9.235672e-09,-0.000049,NaN,NaN,NaN,0,1989-09,1987-05,0,0,0,0,0,0,NaN
3,3,3,10000,1986-05,13007.0,NaN,OPTIMUM MANUFACTURING -CL A,1.284,0.578,0.395,0.706,-0.484,10.0,1989-09-30,-3.10938,-0.222656,3793.0,1.0,0.055326,1.724749e+12,0.000411,0.000041,-0.000599,-0.000060,0.773142,-6.838025e-09,-0.000033,0.718449,NaN,NaN,0,1989-09,1987-05,0,0,0,0,0,0,NaN
4,4,4,10000,1986-06,13007.0,NaN,OPTIMUM MANUFACTURING -CL A,1.284,0.578,0.395,0.706,-0.484,10.0,1989-09-30,-3.09375,-0.005025,3793.0,1.0,0.015744,1.748550e+12,0.000413,0.000041,-0.000602,-0.000060,0.680049,-6.711042e-09,-0.000034,0.520684,NaN,NaN,0,1989-09,1987-05,0,0,0,0,0,0,NaN


## Calculate Returns

In [12]:
import timeit
start = timeit.default_timer()
def returns(PRC):
  returns = PRC.pct_change()
  return returns
  

# Forward all prices
temp2 = df2.set_index(['date_month', 'PERMNO']).copy()['AdjPRC'].groupby(level=1).transform(returns).reset_index().rename(columns={'AdjPRC': 'RET2'})


#df2 = df2.reset_index()
df2 = pd.merge(df2,
               temp2,
               how='inner',
               left_on=['PERMNO', 'date_month'],
               right_on=['PERMNO', 'date_month']
               )
stop = timeit.default_timer()
print('Time: ', stop - start)

Time:  53.11737797699993


## Calculate Log Returns

In [13]:
import timeit
start = timeit.default_timer()
def logReturns(PRC):
  logReturn = np.log(PRC) - np.log(PRC.shift(1))
  return logReturn
  

# Forward all prices
temp2 = df2.set_index(['date_month', 'PERMNO']).copy()['AdjPRC'].groupby(level=1).transform(logReturns).reset_index().rename(columns={'AdjPRC': 'logRET'})


#df2 = df2.reset_index()
df2 = pd.merge(df2,
               temp2,
               how='inner',
               left_on=['PERMNO', 'date_month'],
               right_on=['PERMNO', 'date_month']
               )
stop = timeit.default_timer()
print('Time: ', stop - start)

Time:  45.580563129999746


## Fill Missing Returns with 0

In [19]:
df2['RET2'] = df2['RET2'].fillna(0)
df2['logRET'] = df2['logRET'].fillna(0)

In [20]:
df2[['PERMNO', 'date_month', 'PRC', 'CFACPR', 'AdjPRC', 'RET', 'RET2', 'logRET']][:50]

,PERMNO,date_month,PRC,CFACPR,AdjPRC,RET,RET2,logRET
0,10000,1986-02,-3.25000,1.0,NaN,-0.257143,0.000000,0.000000
1,10000,1986-03,-4.43750,1.0,NaN,0.365385,0.000000,0.000000
2,10000,1986-04,-4.00000,1.0,NaN,-0.098592,0.000000,0.000000
3,10000,1986-05,-3.10938,1.0,NaN,-0.222656,0.000000,0.000000
4,10000,1986-06,-3.09375,1.0,NaN,-0.005025,0.000000,0.000000
5,10000,1986-07,-2.84375,1.0,NaN,-0.080808,0.000000,0.000000
6,10000,1986-08,-1.09375,1.0,NaN,-0.615385,0.000000,0.000000
7,10000,1986-09,-1.03125,1.0,NaN,-0.057143,0.000000,0.000000
8,10000,1986-10,-0.78125,1.0,NaN,-0.242424,0.000000,0.000000
9,10000,1986-11,-0.82813,1.0,NaN,0.060000,0.000000,0.000000


In [21]:
df2.to_csv('Data/testNEWDF.csv')

Unnamed: 0                                  0
PERMNO                                      0
date_month                                  0
GVKEY                                  499136
cik                                    652927
conm                                   499136
atq                                    702966
ceqq                                   638973
cheq                                   715679
ltq                                    710403
niq                                    519483
dlrsn                                 1195857
dldte                                 1195857
PRC                                         0
RET                                         0
SHROUT                                      0
CFACPR                                      0
vwretdSP500                                 0
totvalSP500                                 0
NITA                                   716536
NIMTA                                  714826
TLTA                              

In [71]:
test.head()

,,Unnamed: 0,GVKEY,cik,conm,atq,ceqq,cheq,ltq,niq,dlrsn,dldte,PRC,RET,SHROUT,CFACPR,vwretdSP500,totvalSP500,NITA,NIMTA,TLTA,TLMTA,EXRET,RSIZE,CASHMTA,SIGMA,effectiveDate_Bloomberg_date_month,fileDate_AuditAnalytics_month,bankruptcyIndicator,bankruptcyDate,last_date_existance,bankruptFirm,bankruptcyWithin3Months,bankruptcyWithin6Months,bankruptcyWithin12Months,bankruptcyWithin24Months,bankruptcyWithin60Months,AdjPRC
date_month,PERMNO,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1986-02,10000,0,13007.0,NaN,OPTIMUM MANUFACTURING -CL A,1.265,0.689,0.743,0.576,-0.132,10.0,1989-09-30,-3.25000,-0.257143,3680.0,1.0,0.076525,1.589623e+12,0.000110,0.000011,-0.000482,-0.000048,NaN,-7.523798e-09,-0.000062,NaN,NaN,NaN,0,1989-09,1987-05,0,0,0,0,0,0,-3.25000
1986-03,10000,1,13007.0,NaN,OPTIMUM MANUFACTURING -CL A,1.265,0.689,0.743,0.576,-0.132,10.0,1989-09-30,-4.43750,0.365385,3680.0,1.0,0.055832,1.671574e+12,0.000081,0.000008,-0.000353,-0.000035,0.495175,-9.769236e-09,-0.000046,NaN,NaN,NaN,0,1989-09,1987-05,0,0,0,0,0,0,-4.43750
1986-04,10000,2,13007.0,NaN,OPTIMUM MANUFACTURING -CL A,1.265,0.689,0.743,0.576,-0.132,10.0,1989-09-30,-4.00000,-0.098592,3793.0,1.0,-0.013348,1.642761e+12,0.000087,0.000009,-0.000380,-0.000038,0.759830,-9.235672e-09,-0.000049,NaN,NaN,NaN,0,1989-09,1987-05,0,0,0,0,0,0,-4.00000
1986-05,10000,3,13007.0,NaN,OPTIMUM MANUFACTURING -CL A,1.284,0.578,0.395,0.706,-0.484,10.0,1989-09-30,-3.10938,-0.222656,3793.0,1.0,0.055326,1.724749e+12,0.000411,0.000041,-0.000599,-0.000060,0.773142,-6.838025e-09,-0.000033,0.718449,NaN,NaN,0,1989-09,1987-05,0,0,0,0,0,0,-3.10938
1986-06,10000,4,13007.0,NaN,OPTIMUM MANUFACTURING -CL A,1.284,0.578,0.395,0.706,-0.484,10.0,1989-09-30,-3.09375,-0.005025,3793.0,1.0,0.015744,1.748550e+12,0.000413,0.000041,-0.000602,-0.000060,0.680049,-6.711042e-09,-0.000034,0.520684,NaN,NaN,0,1989-09,1987-05,0,0,0,0,0,0,-3.09375


In [80]:
#test['AdjPRC'].groupby(level=1).value_counts()
# Replace all negative prices with 0
df2.loc[df2['AdjPRC']<0, 'AdjPRC'] = 0

# Replace all 0 prices with NaN
df2.loc[df2['AdjPRC']==0, 'AdjPRC'] = np.nan

test = df2.set_index(['date_month', 'PERMNO'])

test2 = test['AdjPRC'].groupby(level=1).transform(lambda AdjPRC: AdjPRC.ffill()).reset_index()
print(df2.isna().sum())
print(test2.isna().sum())
test2.head()

Unnamed: 0                                  0
PERMNO                                      0
date_month                                  0
GVKEY                                  499136
cik                                    652927
conm                                   499136
atq                                    702966
ceqq                                   638973
cheq                                   715679
ltq                                    710403
niq                                    519483
dlrsn                                 1195857
dldte                                 1195857
PRC                                         0
RET                                         0
SHROUT                                      0
CFACPR                                      0
vwretdSP500                                 0
totvalSP500                                 0
NITA                                   716536
NIMTA                                  714826
TLTA                              

,date_month,PERMNO,AdjPRC
0,1986-02,10000,NaN
1,1986-03,10000,NaN
2,1986-04,10000,NaN
3,1986-05,10000,NaN
4,1986-06,10000,NaN


In [82]:
df2 = df2.reset_index()
df2 = df2.drop(['AdjPRC'], axis=1)
df3 = pd.merge(df2,
               test2,
               how='inner',
               left_on=['PERMNO', 'date_month'],
               right_on=['PERMNO', 'date_month']
               )
df3.head()

,index,Unnamed: 0,PERMNO,date_month,GVKEY,cik,conm,atq,ceqq,cheq,ltq,niq,dlrsn,dldte,PRC,RET,SHROUT,CFACPR,vwretdSP500,totvalSP500,NITA,NIMTA,TLTA,TLMTA,EXRET,RSIZE,CASHMTA,SIGMA,effectiveDate_Bloomberg_date_month,fileDate_AuditAnalytics_month,bankruptcyIndicator,bankruptcyDate,last_date_existance,bankruptFirm,bankruptcyWithin3Months,bankruptcyWithin6Months,bankruptcyWithin12Months,bankruptcyWithin24Months,bankruptcyWithin60Months,AdjPRC
0,0,0,10000,1986-02,13007.0,NaN,OPTIMUM MANUFACTURING -CL A,1.265,0.689,0.743,0.576,-0.132,10.0,1989-09-30,-3.25000,-0.257143,3680.0,1.0,0.076525,1.589623e+12,0.000110,0.000011,-0.000482,-0.000048,NaN,-7.523798e-09,-0.000062,NaN,NaN,NaN,0,1989-09,1987-05,0,0,0,0,0,0,NaN
1,1,1,10000,1986-03,13007.0,NaN,OPTIMUM MANUFACTURING -CL A,1.265,0.689,0.743,0.576,-0.132,10.0,1989-09-30,-4.43750,0.365385,3680.0,1.0,0.055832,1.671574e+12,0.000081,0.000008,-0.000353,-0.000035,0.495175,-9.769236e-09,-0.000046,NaN,NaN,NaN,0,1989-09,1987-05,0,0,0,0,0,0,NaN
2,2,2,10000,1986-04,13007.0,NaN,OPTIMUM MANUFACTURING -CL A,1.265,0.689,0.743,0.576,-0.132,10.0,1989-09-30,-4.00000,-0.098592,3793.0,1.0,-0.013348,1.642761e+12,0.000087,0.000009,-0.000380,-0.000038,0.759830,-9.235672e-09,-0.000049,NaN,NaN,NaN,0,1989-09,1987-05,0,0,0,0,0,0,NaN
3,3,3,10000,1986-05,13007.0,NaN,OPTIMUM MANUFACTURING -CL A,1.284,0.578,0.395,0.706,-0.484,10.0,1989-09-30,-3.10938,-0.222656,3793.0,1.0,0.055326,1.724749e+12,0.000411,0.000041,-0.000599,-0.000060,0.773142,-6.838025e-09,-0.000033,0.718449,NaN,NaN,0,1989-09,1987-05,0,0,0,0,0,0,NaN
4,4,4,10000,1986-06,13007.0,NaN,OPTIMUM MANUFACTURING -CL A,1.284,0.578,0.395,0.706,-0.484,10.0,1989-09-30,-3.09375,-0.005025,3793.0,1.0,0.015744,1.748550e+12,0.000413,0.000041,-0.000602,-0.000060,0.680049,-6.711042e-09,-0.000034,0.520684,NaN,NaN,0,1989-09,1987-05,0,0,0,0,0,0,NaN
